<img src="../img/carlos_portrait.jpg" width=150 align="right">

# 1.- Introducción al Machine Learning

Autor: Carlos Moreno Morera

Contacto: carmor06@ucm.es

Última revisión por Sergio Benito Martín: 31/05/2025

# 1.3 - Tecnicas de selección de datos

En este Notebook vamos a implementar las ***4 técnicas de selección de datos***; es decir, de que manera se puede dividir el dataset inicial en conjuntos de datos de entrenamiento y test. Para ello utilizaremos la librería de ***Scikit-Learn***.

Para evaluar los modelos obtenidos tras la aplicación de alguna de las técnicas de ML, es necesario disponer de un ***conjunto de datos*** (etiquetados o no) para ***generar el mejor modelo  posible y minimizar el error empírico***.

Dado un conjunto de datos, podemos enumerar los siguientes ***métodos de selección de datos***:
<span></span><br><br>
    - **[Resustitución](#M0)**: Todos los datos disponibles se utilizan como datos de test y de entrenamiento.
<span></span><br><br>
    - **[Partición (Hold Out)](#M1)**: Divide los datos en dos subconjuntos: uno de entrenamiento y uno de test.
<span></span><br><br>
    - **[Validación cruzada (Cross Validation)](#M2)**: Divide los datos aleatoriamente en ‘N’ bloques. Cada bloque se utiliza como test para un sistema entrenado por el resto de bloques.
<span></span><br><br>
    - **[Exclusión individual (Leave One Out)](#M3)**: Este método utiliza cada dato individual como dato único de test de un sistema entrenado con todos los datos excepto el de test.

## Ejemplos


Para ver ejemplos de estas técnicas de evaluación supongamos el siguiente conjunto de datos donde:

- ***(i)***: índice del conjunto de datos.
- ***X<sub>1</sub> y X<sub>2</sub>***: Variables de entrada.
- ***y***: Variable de salida.


|(i)|X<sub>1</sub>|X<sub>2</sub>|y|
|---|---|---|---|
|0|0|1|1|
|1|1|1|4|
|2|1|2|6|
|3|2|1|7|
|4|2|2|9|
|5|3|1|10|
|6|3|2|12|
|7|3|3|14|
|8|1|3|8|
|9|2|3|11|


En este ejemplo vamos a tener ***10 elementos*** en nuestro conjundo de datos y los vamos a pasar a un ***array de numpy***:

In [2]:
import numpy as np

X = np.array([[0,1],[1,1],[1,2],[2,1],[2,2],[3,1],[3,2],[3,3],[1,3],[2,3]])
y = np.array([1,4,6,7,9,10,12,14,8,11])

np.random.seed(42)
X

array([[0, 1],
       [1, 1],
       [1, 2],
       [2, 1],
       [2, 2],
       [3, 1],
       [3, 2],
       [3, 3],
       [1, 3],
       [2, 3]])

In [6]:
y

array([ 1,  4,  6,  7,  9, 10, 12, 14,  8, 11])

## <a name="M0">1. Resustitución</a>

Para el caso de la resustitución es muy sencilla; ***todos los datos del Dataset sirven para el Entrenamiento y el Test***. A modo didáctico realizamos la siguiente asignación:

In [9]:
X_train = X
y_train = y

# modelo Algoritmo_de_Aprendizaje.entrenar(X_train, y_train)

X_test = X
y_test = y

# evaluacion = modelo.evaluar(X_test, y_test)

## <a name="M1">2. Hold Out</a>

La técnica de evaluación ***Hold Out*** divide el Dataset en un ***conjunto de datos de entrenamiento*** y otro ***conjunto de datos de test***, seleccionados a priori de manera aleatoria.


<img src="../img/tecnicas_seleccion/tsl_particion.png" style="width: 500px;"/>

Implementémoslo:

In [10]:
def particion(X, y, porcentaje_test):
    num_test = round(len(X)*porcentaje_test)
    indices_test = np.random.choice(X.shape[0], num_test, replace=False) # replace sirve para indicar si se pueden coger los mismos indices
    X_train = np.delete(X, indices_test, 0)
    y_train = np.delete(y, indices_test, 0)

    X_test = X[indices_test]
    y_test = y[indices_test]
    return X_train, y_train, X_test, y_test

np.random.seed(42)
X_train, y_train, X_test, y_test = particion(X, y, 0.2)

In [11]:
X_train

array([[0, 1],
       [1, 2],
       [2, 1],
       [2, 2],
       [3, 1],
       [3, 2],
       [3, 3],
       [2, 3]])

In [12]:
y_train

array([ 1,  6,  7,  9, 10, 12, 14, 11])

En Scikit utilizamos la función ***train_test_split(X,y,test_size)*** para que nos divida en dos conjuntos el Dataset, indicando como parámetro el porcentaje de datos de test que quermos obtener:

In [13]:
from sklearn.model_selection import train_test_split

X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.2)

print(X_train)
print(y_train)
print(X_test)
print(y_test)

# modelo Algoritmo_de_Aprendizaje.entrenar(X_train, y_train)

# evaluacion = modelo.evaluar(X_test, y_test)

[[1 3]
 [3 1]
 [2 1]
 [2 2]
 [3 3]
 [2 3]
 [3 2]
 [1 2]]
[[0 1]
 [1 1]]
[ 8 10  7  9 14 11 12  6]
[1 4]


## <a name="M2">3. Cross Validation</a>

La técnica del ***Cross Validation, divide el Dataset en 'N' conjuntos*** (o Folds como Scikit lo llama). Para cada uno de los ***'N'*** casos se utiliza un conjunto como datos de test y el resto de conjuntos como datos de entrenamiento. Esta técnica tiene sentido aplicarla cuando se quiere evaluar el modelo generado ***'N'*** veces.


<img src="../img/tecnicas_seleccion/tsl_cv.png" style="width: 600px;"/>

Comencemos implementando la validación cruzada de K-iteraciones:

In [5]:
def cv_k_iterations(X, y, k):
    block_size = len(X)//k

    if len(X) % k > 0:
        block_size+=1

    indices = list(np.random.choice(X.shape[0], len(X), replace=False))
    test_indices = [indices[i*block_size:(i+1)*block_size] for i in range(k)]
    train_indices = [(indices[:i*block_size] + indices[(i+1)*block_size:]) for i in range(k)]
    return list(zip(train_indices, test_indices))

print("Indices Train - Indices Test")
for train, test in cv_k_iterations(X, y, 4):
    print("{} - {}".format(train, test))

Indices Train - Indices Test
[0, 7, 2, 9, 4, 3, 6] - [8, 1, 5]
[8, 1, 5, 9, 4, 3, 6] - [0, 7, 2]
[8, 1, 5, 0, 7, 2, 6] - [9, 4, 3]
[8, 1, 5, 0, 7, 2, 9, 4, 3] - [6]


A continuación, veamos cómo implementar la validación cruzada aleatoria:

In [9]:
def cv_aleatoria(X, y, k, size):
    indices = np.array(list(range(X.shape[0])))
    test_indices = [list(np.random.choice(X.shape[0], size, replace=False)) for i in range(k)]
    train_indices = [list(np.delete(indices, test_indices[i], 0)) for i in range(k)]
    return list(zip(train_indices, test_indices))

print("Indices Train - Indices Test")
for train, test in cv_aleatoria(X, y, 4, 3):
    print("{} - {}".format(train, test))

Indices Train - Indices Test
[0, 2, 3, 4, 6, 7, 9] - [8, 1, 5]
[2, 3, 4, 5, 6, 7, 9] - [0, 1, 8]
[1, 3, 4, 5, 6, 7, 8] - [9, 2, 0]
[0, 2, 3, 4, 5, 8, 9] - [1, 7, 6]


En Scikit utilizamos la clase ***KFold(n_splits)*** para dividir el Dataset en ***'N'*** (n_splits) conjuntos. En este caso no vamos a obtener unos arrays con los datos de entrenamiento y test; si no, los ***índices de los elementos del Dataset que en cada paso actuarán como entrenamiento y como test***:

In [3]:
from sklearn.model_selection import KFold

k_fold = KFold(n_splits=4)
print("Indices Train - Indices Test")
for train, test in k_fold.split(X,y):
    print("{} - {}".format(train, test))

Indices Train - Indices Test
[3 4 5 6 7 8 9] - [0 1 2]
[0 1 2 6 7 8 9] - [3 4 5]
[0 1 2 3 4 5 8 9] - [6 7]
[0 1 2 3 4 5 6 7] - [8 9]


In [4]:
# Dividimos el dataset en 5 conjuntos de datos
k_fold = KFold(n_splits=4)

# Ejemplo de como accedemos a los elementos de la variable de salida y
for train, test in k_fold.split(X, y):
    print("y_train: {}".format(y[train]))
    # modelo = Algoritmo_de_Aprendizaje.entrenar(X[train], y[train])

    print("y_test: {}".format(y[test]))
    # evaluar_modelo = modelo.evaluar(X[test], y[test])

y_train: [ 7  9 10 12 14  8 11]
y_test: [1 4 6]
y_train: [ 1  4  6 12 14  8 11]
y_test: [ 7  9 10]
y_train: [ 1  4  6  7  9 10  8 11]
y_test: [12 14]
y_train: [ 1  4  6  7  9 10 12 14]
y_test: [ 8 11]


## <a name="M3">4. Leave One Out</a>

La técnica del ***Leave One One*** es una técnica similar al Cross Validation, pero en este caso el Dataset se ***divide en tantos conjuntos como elementos tenga el Dataset***. Para cada uno de los ***'M'*** elemetos se utiliza un elemento como dato de test y el resto de elementos como datos de entrenamiento. Esta ***técnica es muy costosa de aplicar*** ya que se van a tener tantas iteracciones como elementos tenga el Dataset (***'M'*** veces). Puede tener sentido aplicar esta técnica cuando se tenga un Dataset pequeño y necesitemos evaluar muy a fondo el modelo.


<img src="../img/tecnicas_seleccion/tsl_leave1out.png" style="width: 600px;"/>

In [7]:
def leave_one_out(X, y):
    return cv_k_iterations(X, y, len(X))

print("Indices Train - Indices Test")
for train, test in leave_one_out(X,y):
    print("{} - {}".format(train, test))

Indices Train - Indices Test
[1, 8, 5, 3, 4, 7, 9, 6, 2] - [0]
[0, 8, 5, 3, 4, 7, 9, 6, 2] - [1]
[0, 1, 5, 3, 4, 7, 9, 6, 2] - [8]
[0, 1, 8, 3, 4, 7, 9, 6, 2] - [5]
[0, 1, 8, 5, 4, 7, 9, 6, 2] - [3]
[0, 1, 8, 5, 3, 7, 9, 6, 2] - [4]
[0, 1, 8, 5, 3, 4, 9, 6, 2] - [7]
[0, 1, 8, 5, 3, 4, 7, 6, 2] - [9]
[0, 1, 8, 5, 3, 4, 7, 9, 2] - [6]
[0, 1, 8, 5, 3, 4, 7, 9, 6] - [2]


En Scikit utilizamos la clase ***LeaveOneOut()*** para dividir el Dataset en tantos conjuntos como elementos tenga. En este caso no vamos a obtener unos arrays con los datos de entrenamiento y test; si no, los ***índices de los elementos del Dataset que en cada paso actuarán como entrenamiento y como test***.

In [10]:
from sklearn.model_selection import LeaveOneOut

print("Indices Train - Indices Test")
for train, test in LeaveOneOut().split(X):
    print("{} - {}".format(train, test))

Indices Train - Indices Test
[1 2 3 4 5 6 7 8 9] - [0]
[0 2 3 4 5 6 7 8 9] - [1]
[0 1 3 4 5 6 7 8 9] - [2]
[0 1 2 4 5 6 7 8 9] - [3]
[0 1 2 3 5 6 7 8 9] - [4]
[0 1 2 3 4 6 7 8 9] - [5]
[0 1 2 3 4 5 7 8 9] - [6]
[0 1 2 3 4 5 6 8 9] - [7]
[0 1 2 3 4 5 6 7 9] - [8]
[0 1 2 3 4 5 6 7 8] - [9]
